### RapidFire AI Tutorial Use Case: SFT for Customer Support Q&A Chatbot

##### Note: Minimum hardware required for this notebook is 4*A10 GPUs. Heavily downsampled data for demo purposes

In [ ]:
from rapidfireai import Experiment
from rapidfireai.automl import List, RFGridSearch, RFModelConfig, RFLoraConfig, RFSFTConfig

### Load Dataset and Specify Train and Eval Partitions

In [ ]:
from datasets import load_dataset

dataset=load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

train_dataset=dataset["train"].select(range(320))
eval_dataset=dataset["train"].select(range(320,328))
train_dataset=train_dataset.shuffle(seed=42)
eval_dataset=eval_dataset.shuffle(seed=42)

### Define Data Processing Function

In [ ]:
def sample_formatting_function(row):
    """Function to preprocess each example from dataset"""
    # Special tokens for formatting
    SYSTEM_PROMPT = "You are a helpful and friendly customer support assistant. Please answer the user's query to the best of your ability."
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": row["instruction"]},
            
        ],
        "completion": [
            {"role": "assistant", "content": row["response"]}
        ]
    }

### Initialize Experiment

In [ ]:
# Every experiment instance must be uniquely named
experiment = Experiment(experiment_name="exp1-chatqa-fsdp77")

### Define Custom Eval Metrics Function

In [ ]:
def sample_compute_metrics(eval_preds):  
    """Optional function to compute eval metrics based on predictions and labels"""
    predictions, labels = eval_preds

    # Standard text-based eval metrics: Rouge and BLEU
    import evaluate
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")

    rouge_output = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)
    rouge_l = rouge_output["rougeL"]
    bleu_output = bleu.compute(predictions=predictions, references=labels)
    bleu_score = bleu_output["bleu"]

    return {
        "rougeL": round(rouge_l, 4),
        "bleu": round(bleu_score, 4),
    }

### Define Multi-Config Knobs for Model, LoRA, and SFT Trainer using RapidFire AI Wrapper APIs

In [ ]:
# 2 LoRA PEFT configs lite with different adapter capacities
peft_configs_lite = List([
    RFLoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"], 
        bias="none"
    ),
    RFLoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj"], 
        bias="none"
    ),
])

# 2 combinations in total
config_set_lite = List([
    RFModelConfig(
        model_name="Qwen/Qwen3-32B",  # 32B model
        peft_config=peft_configs_lite,
        training_args=RFSFTConfig(
            learning_rate=1e-4,  
            lr_scheduler_type="linear",
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            # eval_accumulation_steps=4,
            num_train_epochs=1,
            gradient_accumulation_steps=4,   
            logging_steps=1,
            eval_strategy="steps",
            eval_steps=6,
            gradient_checkpointing=True,
            gradient_checkpointing_kwargs={"use_reentrant": False},
            bf16=True,
            tf32=True,
            max_length=256,
            fsdp="full_shard auto_wrap",
            fsdp_config={"backward_prefetch": "backward_pre","forward_prefetch": False,"use_orig_params": False,  "cpu_ram_efficient_loading": True,"offload_params": True, "sync_module_states": True,"limit_all_gathers": True, "sharding_strategy": "FULL_SHARD",
                "auto_wrap_policy": "TRANSFORMER_BASED_WRAP"},
            fsdp_transformer_layer_cls_to_wrap="Qwen3DecoderLayer",   
        ),
        model_type="causal_lm",
        model_kwargs={"device_map":None, "torch_dtype": "bfloat16", "use_cache": False},
        formatting_func=sample_formatting_function,
        compute_metrics = sample_compute_metrics,
        generation_config={
            "max_new_tokens": 128,
            "do_sample": False,
            "use_cache": True,
        }

    )
])


#### Define Model Creation Function for All Model Types Across Configs

In [ ]:

def sample_create_model(model_config): 
     """Function to create model object for any given config; must return tuple of (model, tokenizer)"""
     from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, GptOssForCausalLM

     model_name = model_config["model_name"]
     model_type = model_config["model_type"]
     model_kwargs = model_config["model_kwargs"]
     
 
     if model_type == "causal_lm":
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "gpt":                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
          model = GptOssForCausalLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "seq2seq_lm":
          model = AutoModelForSeq2SeqLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "masked_lm":
          model = AutoModelForMaskedLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "custom":
          # Handle custom model loading logic, e.g., loading your own checkpoints
          # model = ... 
          pass
     else:
          # Default to causal LM
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
      
     tokenizer = AutoTokenizer.from_pretrained(model_name)
     model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
     return (model,tokenizer)

#### Generate Config Group

In [ ]:
# Simple grid search across all sets of config knob values = 4 combinations in total
config_group = RFGridSearch(
    configs=config_set_lite,
    trainer_type="SFT"
)

### Run Multi-Config Training

In [ ]:
# Launch training of all configs in the config_group with swap granularity of 4 chunks
experiment.run_fit(config_group, sample_create_model, train_dataset, eval_dataset, num_chunks=4,seed=42, num_gpus=4)

### End Current Experiment

In [ ]:
experiment.end()